In [12]:
from config import *
import requests
import pandas as pd
from chatbot import chat
from message_sender import sender

pd.set_option('display.max_columns',False)  
pd.set_option('display.max_rows',False)

In [13]:
df=pd.read_excel(data_path) 

In [ ]:
import time
import random
import phonenumbers
import json 

results = []

for idx, row in df.iterrows():
    try:
        phone_str = str(row['Phone']).strip()
        name = row['Name']
        
        # Add + prefix if not present
        if not phone_str.startswith('+'):
            phone_str = '+' + phone_str
        
        # Try to parse phone number automatically
        try:
            # Parse phone number (must include country code)
            parsed = phonenumbers.parse(phone_str, None)

            phone = phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164)
        except:
            # If parsing fails, keep original
            phone = phone_str
        
        # Create user prompt with current lead data
        user_prompt = f"{row}"
        
        # Generate message
        message = chat(user_prompt)
        # Send message
        sender(phone, message)
        
        results.append({
            'Name': name,
            'Phone': phone,
            'Status': 'Sent',
            'Message': message[:50] + '...'
        })
        
        print(f"✓ Sent to {name} ({phone})")
        
    except Exception as e:
        results.append({
            'Name': row['Name'],
            'Phone': row['Phone'],
            'Status': 'Failed',
            'Message': str(e)
        })
        print(f"✗ Failed to send to {row['Name']}: {e}")

# Display results summary
results_df = pd.DataFrame(results)
print(f"\n\nSummary:")
print(f"Total: {len(results_df)}")
print(f"Sent: {len(results_df[results_df['Status'] == 'Sent'])}")
print(f"Failed: {len(results_df[results_df['Status'] == 'Failed'])}")